# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged: "home" if they are below 30% risk and the recommended hospital if they are at or above 30%.

Each step in the midterm will build up to form your complete solution. 

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [1]:
def priest(sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status):
    """(str, int, int, float, int, init, , float, float, str, str, str)->float
    Using the provided algorithm, this function computes the PRIEST
    COVID Clinical Severity Score.  This score is transplated using the provided
    table to compute a 30-day probability of adverse outcomes, which
    the function returns.
    
    See https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence for 
    further documentation.

    >>> priest('female', 16, 1, 0.99, 41, 91, 95, 'alert', 'air', 'unrestricted normal activity')[0]
    0.22

    >>> priest('male',34,12,0.97,65,130,37.0,'alert','air','unrestricted normal activity')[0]
    0.01

    >>> priest('male',68,12,0.97,65,130,37.0,'alert','air','unrestricted normal activity')[0]
    0.03

    >>> priest('gunk',34,12,0.97,65,130,37.0,'alert','air','unrestricted normal activity')[0]
    
    """
    
    result = True

    if sex.lower() == 'female' or sex.lower() == 'f':
        pts_sex = 0
    elif sex.lower() == 'male' or sex.lower() == 'm':
        pts_sex = 1
    else:
        pts_sex = -99
        result = None
        
    if age < 16:
        return None
    elif age <= 49:
        pts_age = 0
    elif age <= 65:
        pts_age = 2
    elif age <= 80:
        pts_age = 3
    else:
        pts_age = 4
        
    if breath <9:
        pts_breath = 3
    elif breath <= 11:
        pts_breath = 1
    elif breath <= 20:
        pts_breath = 0
    elif breath <= 24:
        pts_breath = 2
    else:
        pts_breath = 3
        
    if o2sat > 0.95:
        pts_o2 = 0
    elif o2sat >= 0.94:
        pts_o2 = 1
    elif o2sat >= 0.92:
        pts_o2 = 2
    else:
        pts_o2 = 3
    
    if heart < 41:
        pts_heart = 3
    elif heart <= 50:
        pts_heart = 1
    elif heart <= 90:
        pts_heart = 0
    elif heart <= 110:
        pts_heart = 1
    elif heart <= 130:
        pts_heart = 2
    else:
        pts_heart = 3
        
    if systolic < 91:
        pts_systolic = 3
    elif systolic <= 100:
        pts_systolic = 2
    elif systolic <= 110:
        pts_systolic = 1
    elif systolic <= 219:
        pts_systolic = 0
    else:
        pts_systolic = 3
        
    if temp < 35.1:
        pts_temp = 3
    elif temp <= 36.0:
        pts_temp = 1
    elif temp <= 38:
        pts_temp = 0
    elif temp <= 39:
        pts_temp = 1
    else:
        pts_temp = 2
        
    if alertness.lower() == 'alert':
        pts_alertness = 0
    elif alertness.lower() == 'confused or not alert':
        pts_alertness = 3
    else:
        pts_alertness = -99
        result = None
            
    if inspired.lower() == 'air':
        pts_inspired = 0
    elif inspired.lower() == 'supplemental oxygen':
        pts_inspired = 2
    else:
        pts_inspired = -99
        result = None
        
    if status.lower() == 'unrestricted normal activity':
        pts_status = 0
    elif status.lower() == 'limited strenuous activity, can do light activity':
        pts_status = 1
    elif status.lower() == 'limited activity, can self-care':
        pts_status = 2
    elif status.lower() == 'limited self-care':
        pts_status = 3
    elif status.lower() == 'bed/chair bound, no self-care':
        pts_status = 4
    else:
        pts_status = -99
        result = None

    if result is None:
        score = None
    else:
        score = pts_sex + pts_age + pts_breath + pts_o2 + pts_heart + pts_systolic + pts_temp + pts_alertness + pts_inspired + pts_status
        
    score_map = {
        0: 0.01,
        1: 0.01,
        2: 0.02,
        3: 0.02,
        4: 0.03,
        5: 0.09,
        6: 0.15,
        7: 0.18,
        8: 0.22,
        9: 0.26,
        10: 0.29,
        11: 0.34,
        12: 0.38,
        13: 0.46,
        14: 0.47,
        15: 0.49,
        16: 0.55
    }
    
        
    if score is None:
        risk = None
    elif score >= 26:
        risk = 0.99
    elif score >= 17:
        risk = 0.59
    else:
        risk = score_map[score]
        
    return (
        risk, score,
        [pts_sex, pts_age, pts_breath, pts_o2, pts_heart, pts_systolic, pts_temp, pts_alertness, pts_inspired, pts_status],
    )


In [2]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest('female', 16, 1, 0.99, 41, 91, 95, 'alert', 'air', 'unrestricted normal activity')[0]
Expecting:
    0.22
ok
Trying:
    priest('male',34,12,0.97,65,130,37.0,'alert','air','unrestricted normal activity')[0]
Expecting:
    0.01
ok
Trying:
    priest('male',68,12,0.97,65,130,37.0,'alert','air','unrestricted normal activity')[0]
Expecting:
    0.03
ok
Trying:
    priest('gunk',34,12,0.97,65,130,37.0,'alert','air','unrestricted normal activity')[0]
Expecting nothing
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [3]:
import requests
import json

def find_hospital(age, sex, risk):
    """(int, str, float)->str
    Use the professor's REST web service to lookup and return the
    name of the hospital where this person should be sent.
    
    >>> find_hospital(40, 'male', 0.1)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(40, 'female', 0.4)
    'Emory Dunwoody Medical Center'
    """
    
    url = f'https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={age}&sex={sex}&risk_pct={risk}'
    
    try:
        r = requests.get(url)
        info = r.json()
        hospital = info.get('hospital')
    except:
        hospital = None
    
    return hospital

In [4]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(40, 'male', 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(40, 'female', 0.4)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [5]:
import requests
import json

def get_address(hospital):
    """(str)->str
    Fetches the hospital address from the known webpage.
    
    >>> get_address('Southwest Hospital and Medical Center')
    '501 FAIRBURN ROAD SW'
    """
    
    if hospital is None:
        return None
    
    r = requests.get('https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-')
    hospitals = json.loads(r.text)

    entry = hospitals.get(hospital.upper())
    result = entry.get('ADDRESS')

    return result
    

In [6]:
import doctest
doctest.run_docstring_examples(get_address, globals(), True)

Finding tests in NoName
Trying:
    get_address('Southwest Hospital and Medical Center')
Expecting:
    '501 FAIRBURN ROAD SW'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes a file name as a parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [7]:
import requests
import csv

def process_people(limit=-1):
    result = {}

    response = requests.get('https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh')
    data = response.text.split('\n')

    for row in data[:limit]:
        (patient, sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status) = row.split('|')

        if (patient != 'patient'):
            age = int(age)
            breath = float(breath)
            o2sat = float(o2sat)
            heart = float(heart)
            systolic = float(systolic)
            temp = float(temp)
            percentile, score, scores = priest(sex, int(age), float(breath), float(o2sat), float(heart), float(systolic), float(temp), alertness, inspired, status)
            hospital = find_hospital(int(age), sex, float(percentile))
            if hospital is None:
                print(f'Unable to find hospital for {sex}, {age}, {score}, {percentile}')
            location = get_address(hospital)
            result[patient] = [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, percentile, hospital, location]
        
    return result

In [9]:
results = process_people()

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [14]:
a_response = requests.get('https://hds5210-data.s3.amazonaws.com/people_results.json')

In [15]:
a_json = a_response.json()

In [19]:
for r_key, r_val in results.items():
    r_hospital_name = r_val[-2]
    r_address = r_val[-1]

    if r_key in a_json:
        a_val = a_json[r_key]
        a_hospital = a_val[-2]
        a_address = a_val[-1]
        if r_hospital_name == a_hospital and r_address == a_address:
            print(f'MATCH! {r_hospital_name} == {a_hospital}, {r_address} == {a_address}')
        else:
            print(f'FAIL!! {r_hospital_name} ?= {a_hospital}, {r_address} ?= {a_address}')
    else:
        print(f'Missing key: {r_key}')

MATCH! Select Specialty Hospital - Northeast Atlanta == Select Specialty Hospital - Northeast Atlanta, 1821 CLIFTON ROAD NE == 1821 CLIFTON ROAD NE
MATCH! Select Specialty Hospital - Northeast Atlanta == Select Specialty Hospital - Northeast Atlanta, 1821 CLIFTON ROAD NE == 1821 CLIFTON ROAD NE
MATCH! Select Specialty Hospital - Northeast Atlanta == Select Specialty Hospital - Northeast Atlanta, 1821 CLIFTON ROAD NE == 1821 CLIFTON ROAD NE
MATCH! Wesley Woods Geriatric Hospital == Wesley Woods Geriatric Hospital, 1821 CLIFTON ROAD, NE == 1821 CLIFTON ROAD, NE
MATCH! Southwest Hospital and Medical Center == Southwest Hospital and Medical Center, 501 FAIRBURN ROAD SW == 501 FAIRBURN ROAD SW
MATCH! Childrens Healthcare of Atlanta at Scottish Rite == Childrens Healthcare of Atlanta at Scottish Rite, 1001 JOHNSON FERRY RD == 1001 JOHNSON FERRY RD
MATCH! Southwest Hospital and Medical Center == Southwest Hospital and Medical Center, 501 FAIRBURN ROAD SW == 501 FAIRBURN ROAD SW
MATCH! Select 

---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---